In [ ]:
%pylab inline 


In [ ]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
from collections import Counter
import time
import progressbar as pb
from tqdm import tqdm
import os
import shutil
from empyrical import sortino_ratio, calmar_ratio, omega_ratio
from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, VecEnv, VecEnvWrapper
from stable_baselines import A2C, PPO2
from stable_baselines.common.vec_env import DummyVecEnv

import progressbar as pb

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, LSTM, Reshape, Dropout, Input, Concatenate
from keras.optimizers import Adam

from rl.agents.cem import CEMAgent 
from rl.agents.ddpg import DDPGAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory, EpisodeParameterMemory
from rl.processors import WhiteningNormalizerProcessor
from rl.agents import DDPGAgent
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess

from trading_env import TradingEnv

In [ ]:
# load the market data
input_source = np.load(open('data_dummy.npy','rb'))
to_predict = np.load(open('data_dummy_targets.npy','rb'))

to_predict = to_predict[3,:].reshape(-1)

input_source = input_source.T

In [ ]:
is_orig = np.copy(input_source)
cp = int(0.8*len(input_source))
test_input_source = input_source[cp:, :]
test_to_predict = to_predict[cp:]
input_source = input_source[0:cp, :]
to_predict = to_predict[0:cp]

In [ ]:
bars_per_episode = 1000
winlen = 1
traded_amt = 1000
commission = 0
slippage = 0.0

In [ ]:
# multiprocess environment
#n_cpu = 16
#env = SubprocVecEnv([lambda: TradingEnv() for i in range(n_cpu)])
env = TradingEnv(input_source, to_predict,
                 winlen=winlen, bars_per_episode=bars_per_episode, traded_amt=traded_amt,
                 commission=commission, slippage=slippage,
                 reward_type='cur_balance',
                 min_ratio_trades = 20,
                 max_position_time = 30,
                 )

In [ ]:
import MultiNEAT as NEAT
import MultiNEAT.viz as viz
import random as rnd
import pickle
import numpy as np
import cv2

rng = NEAT.RNG()
rng.TimeSeed()

params = NEAT.Parameters()
params.PopulationSize = 150
params.DynamicCompatibility = True
params.WeightDiffCoeff = 1.0
params.CompatTreshold = 2.0
params.YoungAgeTreshold = 15
params.SpeciesMaxStagnation = 15
params.OldAgeTreshold = 35
params.MinSpecies = 2
params.MaxSpecies = 4
params.RouletteWheelSelection = False
params.Elitism = True
params.RecurrentProb = 0.15
params.OverallMutationRate = 0.2

params.MutateWeightsProb = 0.8
params.MutateNeuronTimeConstantsProb = 0.1
params.MutateNeuronBiasesProb = 0.1

params.WeightMutationMaxPower = 0.5
params.WeightReplacementMaxPower = 1.0
params.MutateWeightsSevereProb = 0.5
params.WeightMutationRate = 0.25

params.TimeConstantMutationMaxPower = 0.1
params.BiasMutationMaxPower = params.WeightMutationMaxPower

params.MaxWeight = 8

params.MutateAddNeuronProb = 0.1
params.MutateAddLinkProb = 0.2
params.MutateRemLinkProb = 0.0

params.MinActivationA  = 1.0
params.MaxActivationA  = 6.0

params.MinNeuronTimeConstant = 0.04
params.MaxNeuronTimeConstant = 0.24

params.MinNeuronBias = -params.MaxWeight
params.MaxNeuronBias = params.MaxWeight

params.ActivationFunction_SignedSigmoid_Prob = 0.0
params.ActivationFunction_UnsignedSigmoid_Prob = 0.0
params.ActivationFunction_Tanh_Prob = 1.0
params.ActivationFunction_SignedStep_Prob = 0.0
params.ActivationFunction_Linear_Prob = 0.0

params.CrossoverRate = 0.75  # mutate only 0.25
params.MultipointCrossoverRate = 0.4
params.SurvivalRate = 0.2

params.MutateNeuronTraitsProb = 0
params.MutateLinkTraitsProb = 0


trials = 15
render_during_training = 0

g = NEAT.Genome(0, int(np.prod(env.observation_space.shape))+1, 0, env.action_space.n, False,
                    NEAT.ActivationFunction.TANH, NEAT.ActivationFunction.TANH, 0, params, 0)
pop = NEAT.Population(g, params, True, 1.0, rnd.randint(0, 1000))


hof = []
maxf_ever = 0


In [ ]:
def interact_with_nn(observation):
    inp = observation.tolist()
    net.Input(inp + [1.0])
    #print(inp)
    net.ActivateLeaky(0.1)
    out = list(net.Output())
    return out


def do_trial():
    #observation, reward, t, img, action, done, info, avg_reward
    observation = env.reset()
    net.Flush()

    f = 0
    for t in range(bars_per_episode):
        out = interact_with_nn(observation)
        action = np.argmax(out)
        observation, reward, done, info = env.step(action)
        if done: break
        f += reward
    return f/t

try:

    for generation in range(200):
        for i_episode, genome in tqdm(enumerate(NEAT.GetGenomeList(pop))):
            net = NEAT.NeuralNetwork()
            genome.BuildPhenotype(net)
            avg_reward = 0
            for trial in range(trials):
                avg_reward += do_trial()
            avg_reward /= trials
            genome.SetFitness(avg_reward)

        maxf = max([x.GetFitness() for x in NEAT.GetGenomeList(pop)])
        print('Generation: {}, max fitness: {}'.format(generation, maxf))

        if maxf > maxf_ever:
            hof.append(pickle.dumps(pop.GetBestGenome()))
            maxf_ever = maxf

        pop.Epoch()

except KeyboardInterrupt:
    pass


In [ ]:
env = TradingEnv(test_input_source, test_to_predict,
                 winlen=winlen, bars_per_episode=bars_per_episode, traded_amt=traded_amt,
                 commission=commission, slippage=slippage,
                 reward_type='cur_balance',
                 min_ratio_trades = 20,
                 max_position_time = 30,
                 )
# visualize the behavior for one random episode
bars_per_episode = 1000

observation = env.reset()
done = False
navs = []
while not done:
    action = agent.forward(observation)
    observation, reward, done, info = env.step(action)
    navs.append(reward)

kl = []
t = 0
for n in navs:
    t += n
    kl.append(t)
plot(kl);

In [ ]:
# calculate the likelihood of success for any given episode
try:
    # calculate the likelihood of success for any given episode
    l = 100
    krl = []
    p = pb.ProgressBar(max_value=l)
    for i in range(l):
        p.update(i)
        observation = env.reset()
        done = False
        navs = []
        while not done:
            action = agent.forward(observation)
            observation, reward, done, info = env.step(action)
            navs.append(reward)
        krl.append(sum(navs))
    p.finish()
except KeyboardInterrupt:
    pass

krl = np.array(krl)
print('Profit likelihood: %3.3f%%' % (100*(sum(krl > 0) / len(krl))))

hist(krl);